In [ ]:
from db_inference import simple_sql_db, simple_neo4j
from functools import lru_cache

from more_itertools import chunked
from utils import notebook_util
notebook_util.disp_notebook_full_width()


In [ ]:
sql_db = simple_sql_db.SimpleSqlDb()
sql_db.get_tables()

In [ ]:
conn = simple_neo4j.Neo4jConnection()

In [ ]:
# add uniqueness constraints
conn.query('CREATE CONSTRAINT ON (p:P30) ASSERT p.p30 IS UNIQUE')
conn.query('CREATE CONSTRAINT ON (p:P90) ASSERT p.p90 IS UNIQUE')
conn.query('CREATE CONSTRAINT ON (p:P100) ASSERT p.p100 IS UNIQUE')

In [ ]:
cur = sql_db.conn.cursor()
for r in cur.execute(f"SELECT * FROM prot2protwindow LIMIT 1000;"):
    print(r['p1hash'], r['p2hash'])

#     # create node 1
#     cipher_q = "CREATE (n:P100 {p100: '%s'})" % r['p1hash']
#     print(cipher_q)
#     conn.query(cipher_q)

# #     # create node 2
#     cipher_q = "CREATE (n:P100 {p100: '%s'})" % r['p2hash']
#     print(cipher_q)
#     conn.query(cipher_q)
    
    # can't store bidirectional relationship.  
    # https://stackoverflow.com/questions/43783241/why-neo4j-dont-allows-not-directed-or-bidirectional-relationships-at-creation-t/43793964
    cipher_q = "MATCH (n:P100 {p100: '%s'}), (m:P100 {p100: '%s'}) MERGE (n)-[:WINDOWED_NEIGHBOR]->(m)" % (r['p1hash'], r['p2hash'])
    print(cipher_q)
    conn.query(cipher_q)
    

In [ ]:
cur = sql_db.conn.cursor()
rows = [dict(x) for x in cur.execute(f"SELECT * FROM prot2protwindow LIMIT 10000;")]
cur.close()

query = '''
        UNWIND $rows AS row
        MATCH (n:P100 {p100: row.p1hash}), (m:P100 {p100: row.p2hash})
        MERGE (n)-[:WINDOWED_NEIGHBOR]->(m)
        RETURN count(*) as total
        '''
conn.query(query, parameters = {'rows':rows})



In [ ]:
cur = sql_db.conn.cursor()
rows = [dict(x) for x in cur.execute(f"SELECT * FROM clusters LIMIT 1000;")]
cur.close()

query = '''
        UNWIND $rows AS row
        MERGE (c30:P30 {p30: row.p30})
        MERGE (c90:P90 {p90: row.p90})
        MERGE (c100:P100 {p100: row.p100})
        MERGE (c30)-[:P30_CLUSTERING]->(c90)
        MERGE (c90)-[:P90_CLUSTERING]->(c100)
        RETURN count(*) as total
        '''
conn.query(query, parameters = {'rows':rows})

In [ ]:
%%time

cur = sql_db.conn.cursor()
cur.execute(f"SELECT * FROM prot2protwindow LIMIT 1000000;")

for rows in chunked(cur, 1000):x
#     print(len(rows))
    len(rows)

cur.close()